In [1]:
import numpy as np
import pandas as pd 

In [2]:
#importing the training data
imdb_data=pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
import re
from bs4 import BeautifulSoup

def remove_html_tags(text): #remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()
    return clean_text

text = "<p>This is an <b>example</b> text with HTML tags.</p>"
clean_text = remove_html_tags(text)
print(clean_text)

This is an example text with HTML tags.


In [4]:
import string

def remove_punctuation(text):
    clean_text = "".join([char for char in text if char not in string.punctuation])
    return clean_text

text = "Hello! This is an example sentence."
clean_text = remove_punctuation(text)
print(clean_text)


Hello This is an example sentence


In [5]:
import re

def remove_special_characters(text):
    clean_text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return clean_text

text = "Hello! This is @an example #sentence."
clean_text = remove_special_characters(text)
print(clean_text)

Hello This is an example sentence


In [6]:
def normalize_text(text):
    clean_text = text.lower()
    return clean_text

text = "Hello! This is an EXAMPLE sentence."
clean_text = normalize_text(text)
print(clean_text)

hello! this is an example sentence.


In [7]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

text = "I can't believe it!"
expanded_text = expand_contractions(text)
print(expanded_text)

I cannot believe it!


In [9]:
def remove_numbers(text):
    clean_text = re.sub(r"\d+", "", text)
    return clean_text

text = "This is an example sentence with 123 numbers."
clean_text = remove_numbers(text)
print(clean_text)


This is an example sentence with  numbers.


In [10]:
def remove_extra_whitespace(text):
    clean_text = " ".join(text.split())
    return clean_text

text = "   This    is   an   example   sentence.   "
clean_text = remove_extra_whitespace(text)
print(clean_text)


This is an example sentence.


In [11]:
def preprocess_text(text):

    text = remove_html_tags(text)
    text = remove_punctuation(text)
    text = remove_special_characters(text)
    text = normalize_text(text)
    text = expand_contractions(text)
    text = remove_numbers(text)
    text = remove_extra_whitespace(text)

    return text

imdb_data['review'] = imdb_data['review'].apply(preprocess_text)


/tmp/ipykernel_28/3455125503.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [12]:
from transformers import BertTokenizer
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
# Load cleaned reviews 
reviews = imdb_data['review']
# Tokenize each review 
tokenized_reviews = []
for review in reviews:
  encoded = tokenizer.encode_plus(
    review, 
    add_special_tokens=True, # Add [CLS], [SEP]
    truncation=True,
    max_length=128, # Max review length
    pad_to_max_length=True,
    return_attention_mask=True # Return attention mask   
  )
  
  # Add the encoded dictionary to list
  tokenized_reviews.append(encoded)

# Separate input ids and attention masks
input_ids = [r['input_ids'] for r in tokenized_reviews] 
attention_masks = [r['attention_mask'] for r in tokenized_reviews]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [14]:
import torch

In [15]:
imdb_data['sentiment']

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [16]:
labels = imdb_data['sentiment']
# 0 = negative, 1 = positive
labels[labels == 'positive'] = 1 
labels[labels == 'negative'] = 0

labels = torch.tensor(labels)

In [17]:
from sklearn.model_selection import train_test_split

# Train test split 
train_inputs, test_inputs, train_masks, test_masks = train_test_split(input_ids, attention_masks, test_size=0.2) 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
# Split labels using same indexes
train_labels, test_labels = train_test_split(labels, test_size=0.2)

In [19]:
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)

test_inputs = torch.tensor(test_inputs) 
test_masks = torch.tensor(test_masks)

In [20]:
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=16) 

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=16)

In [21]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [22]:
import torch.nn as nn

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

In [23]:

# Get device IDs 
device_ids = [0, 1]

# Move model to GPU
model = model.to('cuda')

# Wrap model  
model = nn.DataParallel(model, device_ids=device_ids)

In [25]:
# Set model to train mode
model.train() 

for epoch in range(2):
  
  for batch in train_dataloader:
    
    # Unpack batch
    
    b_input_ids, b_attn_masks, b_labels = batch

    
    # Zero gradients
    optimizer.zero_grad()   
        
    b_input_ids = b_input_ids.to('cuda') 
    b_attn_masks = b_attn_masks.to('cuda')
    b_labels = b_labels.to('cuda') 

    outputs = model(b_input_ids, attention_mask=b_attn_masks)

    loss = loss_fn(outputs[0], b_labels)

    # Backward pass to get gradients
    loss.backward()

    # Update parameters
    optimizer.step()

In [35]:
torch.save(model, 'model')